In [ ]:
from __future__ import annotations

# How to download 'IONEX' files

In `spinifex` we provide an interface for obtaining ionospheric model data from online sources that host IONEX files. Most users won't need to interact with these interfaces directly, as they will be called 'under the hood' by higher-level functions such as `spinifex.get_rm.get_rm_from_skycoord`.

## Non-IONEX models

We note here that `spinifex` also supports models of the ionosphere that do not actually require IONEX files. For example, you can swap the `iono_model` value to one of the other supported models, such as the International Reference Ionosphere (IRI). The available models can be enumerated as follows (we typically default to `spinifex.ionospheric.ionospheric_models.ionex`):

In [ ]:
from pprint import pprint

from spinifex.ionospheric import ionospheric_models

pprint(ionospheric_models)

## Obtaining IONEX files

### IONEX file servers

Assuming you have elected to use an IONEX-based model, there are few options for obtaining the required files. Spinifex supports three hosts of IONEX data:

- NASA CDDIS - `"cddis"`: https://cddis.nasa.gov/archive/gnss/products/ionex,
- Barcelona Tech UPC 'Champan' - `"chapman"`: http://chapman.upc.es/tomion/rapid,
- IGS Ionosphere Working Group - `"igsiono"`: ftp://igs-final.man.olsztyn.pl",

These services have their own unique ways of naming and storing files, which we have encoded in `spinifex`. These have also historically changed over time. If you find an issue on obtaining a file with `spinifex` which exists on one of these servers, please raise an issue.

#### CDDIS

The CDDIS has a large archive of IONEX data back to the 90's, but now requires authentication to access their data. To use this service you must register a NASA EARTHDATA Account [here](https://urs.earthdata.nasa.gov/). Then you need to create a `~/.netrc` file with the following text
```
machine urs.earthdata.nasa.gov login <username> password <password>
```
where `<username>` and `<password>` should be replaced with the appropriate values matching your account. If using CDDIS, Spinifex will search for this file and raise an error if it does not exist.

#### Chapman and IGSiono

The other IONEX servers have less historical data, but do not require an account to access them. For simplicity we default to using the Chapman service.

### Types of IONEX files

There are various types of IONEX files available. If in doubt, we have attempted to use sensible defaults throughout `spinifex`. Of note, there are various centres around the world the provide their own processing of GPS data. Further, there are different time resolutions available depending on the centre, the date, and the IONEX server.

The supported processing centres and default time resolutions are as follows:

In [ ]:
from spinifex.ionospheric import ionex_download

print("Supported IONEX centres:")
pprint(ionex_download.CENTER_NAMES)
print("Default IONEX time resolutions:")
pprint(ionex_download.DEFAULT_TIME_RESOLUTIONS)

Our main API for downloading IONEX files is `download_ionex`. The user is required to specify the required time as an `astropy.time.Time` object. The user can then optionally chose the a host server name (e.g. `"chapman"`), the IONEX centre prefix (e.g. `"uqr`"), the URL of the host server (more on that below), the time resolution as an `astropy.time.Time` object, the IONEX solution type (i.e. `"final"` or `"rapid"`), and the output directory for IONEX files as a `pathlib.Path`.

In [ ]:
help(ionex_download.download_ionex)

## Offline use

Before downloading, we check the `output_directory` Path for the required files. If located, we skip re-downloading and just point to the existing files.

Additionally, the `server` argument effectively sets the filename formatting to match the given IONEX host, whereas the `url_stem` sets where the file will actually be retrieved from. This can be useful to take advantage of in cases where internet access is not guaranteed, such as HPC environments. For example, users may wish to make a local offline mirror of a host server. We could do this by:

```bash
cd /path/to/our/disk/
wget --recursive --no-parent http://chapman.upc.es/tomion/rapid/
```

We could then point `download_ionex` call to our local files by setting `url_stem` as:

```python
download_ionex(
    ...
    server="chapman",
    url_stem="file:///path/to/our/disk/chapman.upc.es/tomion/rapid"
    ...
)
```